# Processo Seletivo - Visio Infraestrutura de Software
João Pedro Trevisan
Agosto de 2024  
  
---

Disclaimer: algumas células não possuem saídas!  
Esta decisão foi baseada no fato de que a análise inicial foi feita em um arquivo .py fazendo uso das funções criadas e importadas para apoiá-la.  
Contudo, apresentar os dados dentro de um notebook python é uma maneira mais confortável de se trabalhar e se ler o resultado da manipulação de dados.

## Importações, download e recorte dos videos

In [ ]:
from google.cloud import storage
import pandas as pd
import subprocess

from utils.test_reencoder import run_tests_and_save_results, test_reencoder
from utils.meta_fetch import MetaFetch
from reencoder import AV1Reencoder, VP8Reencoder, VP9Reencoder


In [ ]:
client.create_anonymous_client()
client = storage.client.create_anonymous_client()
bucket = client.bucket(bucket_name='psel_video_samples')

blob = bucket.blob('h264.mp4') # repita para h265 e h264
blob.download_to_filename('videos_visio/h264.mp4')
blob = bucket.blob('h264+.mp4') # repita para h265 e h264
blob.download_to_filename('videos_visio/h264.mp4')
blob = bucket.blob('h265.mp4') # repita para h265 e h264
blob.download_to_filename('videos_visio/h265.mp4')

Para facilitar no tempo de conversão, os videos usados são cortes de 30 segundos dos vídeos originais.  
Isso só não é verdade para o encder AV1, mas isso será esclarecido mais adiante.

In [ ]:
# cortando video h264
video_saida = "videos_visio/h264.mp4"
video_entrada = "videos_visio/30s_h264.mp4"
duracao = "00:00:30"
cmd = [
    "ffmpeg",
    "-i", video_entrada,
    "-t", duracao,
    "-c", "copy",
    video_saida
]
subprocess.run(cmd, check=True)

# cortando video h264+
video_saida = "videos_visio/h264+.mp4"
video_entrada = "videos_visio/30s_h264+.mp4"
duracao = "00:00:30"
cmd = [
    "ffmpeg",
    "-i", video_entrada,
    "-t", duracao,
    "-c", "copy",
    video_saida
]
subprocess.run(cmd, check=True)

# cortando video h265
video_saida = "videos_visio/h265.mp4"
video_entrada = "videos_visio/30s_h265.mp4"
duracao = "00:00:30"
cmd = [
    "ffmpeg",
    "-i", video_entrada,
    "-t", duracao,
    "-c", "copy",
    video_saida
]
subprocess.run(cmd, check=True)

## Testes de Conversão

### Testes VP8

In [ ]:
input_video = "videos_visio/30s_h264.mp4"
output_template = "videos_visio/reencod/VP8/output_h264_vp8_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_1 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)


input_video = "videos_visio/30s_h265.mp4"
output_template = "videos_visio/reencod/VP8/output_h265_vp8_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_2 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)


input_video = "videos_visio/30s_h264+.mp4"
output_template = "videos_visio/reencod/VP8/output_h264+_vp8_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_3 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)

df_result = pd.concat([df_results_1, df_results_2, df_results_3], ignore_index=True)
df_result
df_result.to_csv("vp8.csv", index=False)

In [22]:
df_vp8 = pd.read_csv("vp8.csv")
df_vp8['original_codec'] = ['h264'] * 6 + ['h265'] * 6 + ['h264+'] *6 # adicionando manualmente o codec original pois nos videos esta errado
df_vp8['diferenca_percentual_tamanho'] = ((df_vp8['output_size'] - df_vp8['input_size']) / df_vp8['input_size']) * 100
df_vp8

,codec,options,input_size,output_size,evaluation_result,elapsed_time,crf,speed,original_codec,diferenca_percentual_tamanho
0,vp8,"{'crf': 15, 'speed': 4, 'threads': 2}",1842030,1911505,30.492520,16.163413,15,4,h264,3.771654
1,vp8,"{'crf': 15, 'speed': 2, 'threads': 2}",1842030,749977,32.160616,43.575843,15,2,h264,-59.285299
2,vp8,"{'crf': 30, 'speed': 4, 'threads': 2}",1842030,1911505,30.492520,15.600347,30,4,h264,3.771654
3,vp8,"{'crf': 30, 'speed': 2, 'threads': 2}",1842030,749977,32.160616,45.654684,30,2,h264,-59.285299
4,vp8,"{'crf': 50, 'speed': 4, 'threads': 2}",1842030,1911505,30.492520,14.905854,50,4,h264,3.771654
5,vp8,"{'crf': 50, 'speed': 2, 'threads': 2}",1842030,743564,32.135304,44.200587,50,2,h264,-59.633448
6,vp8,"{'crf': 15, 'speed': 4, 'threads': 2}",1471437,1778665,30.451200,14.582265,15,4,h265,20.879453
7,vp8,"{'crf': 15, 'speed': 2, 'threads': 2}",1471437,691894,31.916114,49.724072,15,2,h265,-52.978347
8,vp8,"{'crf': 30, 'speed': 4, 'threads': 2}",1471437,1778665,30.451200,14.623293,30,4,h265,20.879453
9,vp8,"{'crf': 30, 'speed': 2, 'threads': 2}",1471437,691894,31.916114,50.380263,30,2,h265,-52.978347


In [39]:
tam_menor = df_vp8['diferenca_percentual_tamanho'].min()
tam_menor_i = df_vp8['diferenca_percentual_tamanho'].idxmin()
print(f"Maior diminuicao de tamaho: {tam_menor} na linha: {tam_menor_i}")

temp_menor = df_vp8['elapsed_time'].min()
temp_menor_i = df_vp8['elapsed_time'].idxmin()
print(f"Conversão mais rápida: {temp_menor} na linha: {temp_menor_i}")

psnr_maior = df_vp8['evaluation_result'].max()
psnr_maior_i = df_vp8['evaluation_result'].idxmax()
print(f"Maior valor de PSNR: {psnr_maior} na linha: {psnr_maior_i}")

psnr_menor = df_vp8['evaluation_result'].min()
psnr_menor_i = df_vp8['evaluation_result'].idxmin()
print(f"Menor valor de PSNR: {psnr_menor} na linha: {psnr_menor_i}")

Maior diminuicao de tamaho: -59.63344788087056 na linha: 5
Conversão mais rápida: 14.334309339523315 na linha: 14
Maior valor de PSNR: 32.160616 na linha: 1
Menor valor de PSNR: 30.4512 na linha: 6


### Testes VP9

In [ ]:
input_video = "videos_visio/30s_h264.mp4"
output_template = "videos_visio/reencod/VP9/output_h264_vp9_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_1 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)


input_video = "videos_visio/30s_h265.mp4"
output_template = "videos_visio/reencod/VP9/output_h265_vp9_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_2 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)


input_video = "videos_visio/30s_h264+.mp4"
output_template = "videos_visio/reencod/VP9/output_h264+_vp9_crf{crf}_speed{speed}.webm"
crf_values = [15, 30, 50]
speed_values = [2, 4]
df_results_3 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values)

df_result = pd.concat([df_results_1, df_results_2, df_results_3], ignore_index=True)
df_result
df_result.to_csv("vp9.csv", index=False)

In [33]:
df_vp9 = pd.read_csv("vp9.csv")
df_vp9['original_codec'] = ['h264'] * 6 + ['h265'] * 6 + ['h264+'] *6 # adicionando manualmente o codec original pois nos videos esta errado
df_vp9['diferenca_percentual_tamanho'] = ((df_vp9['output_size'] - df_vp9['input_size']) / df_vp9['input_size']) * 100
df_vp9

,codec,options,input_size,output_size,evaluation_result,elapsed_time,crf,speed,original_codec,diferenca_percentual_tamanho
0,vp9,"{'crf': 15, 'speed': 4, 'threads': 2}",1842030,4879147,44.966804,67.517135,15,4,h264,164.878802
1,vp9,"{'crf': 15, 'speed': 2, 'threads': 2}",1842030,4672195,45.032262,80.596506,15,2,h264,153.643806
2,vp9,"{'crf': 30, 'speed': 4, 'threads': 2}",1842030,2632118,43.510837,67.062914,30,4,h264,42.892244
3,vp9,"{'crf': 30, 'speed': 2, 'threads': 2}",1842030,2569960,43.692490,77.714066,30,2,h264,39.517815
4,vp9,"{'crf': 50, 'speed': 4, 'threads': 2}",1842030,840491,39.066017,64.221460,50,4,h264,-54.371481
5,vp9,"{'crf': 50, 'speed': 2, 'threads': 2}",1842030,825874,39.226264,76.787624,50,2,h264,-55.165008
6,vp9,"{'crf': 15, 'speed': 4, 'threads': 2}",1471437,4155697,43.291725,65.987115,15,4,h265,182.424392
7,vp9,"{'crf': 15, 'speed': 2, 'threads': 2}",1471437,4004487,43.332151,79.020004,15,2,h265,172.148043
8,vp9,"{'crf': 30, 'speed': 4, 'threads': 2}",1471437,2262748,42.337003,66.495231,30,4,h265,53.778109
9,vp9,"{'crf': 30, 'speed': 2, 'threads': 2}",1471437,2216712,42.462425,76.764789,30,2,h265,50.649467


In [38]:
tam_menor = df_vp9['diferenca_percentual_tamanho'].min()
tam_menor_i = df_vp9['diferenca_percentual_tamanho'].idxmin()
print(f"Maior diminuicao de tamaho: {tam_menor} na linha: {tam_menor_i}")

temp_menor = df_vp9['elapsed_time'].min()
temp_menor_i = df_vp9['elapsed_time'].idxmin()
print(f"Conversão mais rápida: {temp_menor} na linha: {temp_menor_i}")

psnr_maior = df_vp9['evaluation_result'].max()
psnr_maior_i = df_vp9['evaluation_result'].idxmax()
print(f"Maior valor de PSNR: {psnr_maior} na linha: {psnr_maior_i}")

psnr_menor = df_vp9['evaluation_result'].min()
psnr_menor_i = df_vp9['evaluation_result'].idxmin()
print(f"Menor valor de PSNR: {psnr_menor} na linha: {psnr_menor_i}")

Maior diminuicao de tamaho: -55.16500817033382 na linha: 5
Conversão mais rápida: 63.24983549118042 na linha: 16
Maior valor de PSNR: 45.173358 na linha: 13
Menor valor de PSNR: 38.913672 na linha: 10


### Testes AV1

In [ ]:
input_video = "videos_visio/5s_h264.mp4"
output_template = "videos_visio/reencod/AV1/output_h264_av1_crf{crf}_speed{speed}.mkv"
crf_values = [30]
speed_values = [8]
input_video_info = MetaFetch(input_video)
bitrate = input_video_info.get_bitrate()
df_results_1 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values, bitrate)


input_video = "videos_visio/5s_h265.mp4"
output_template = "videos_visio/reencod/AV1/output_h265_av1_crf{crf}_speed{speed}.mkv"
crf_values = [30]
speed_values = [8]
input_video_info = MetaFetch(input_video)
bitrate = input_video_info.get_bitrate()
df_results_2 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values, bitrate)


input_video = "videos_visio/5s_h264+.mp4"
output_template = "videos_visio/reencod/AV1/output_h264+_av1_crf{crf}_speed{speed}.mkv"
crf_values = [30]
speed_values = [8]
input_video_info = MetaFetch(input_video)
bitrate = input_video_info.get_bitrate()
df_results_3 = run_tests_and_save_results(AV1Reencoder, input_video, output_template, crf_values, speed_values, bitrate)

df_result = pd.concat([df_results_1, df_results_2, df_results_3], ignore_index=True)
df_result
df_result.to_csv("av1.csv", index=False)

In [36]:
df_av1 = pd.read_csv("av1.csv")
df_av1['original_codec'] = ['h264'] + ['h265'] + ['h264+'] # adicionando manualmente o codec original pois nos videos esta errado
df_av1['diferenca_percentual_tamanho'] = ((df_av1['output_size'] - df_av1['input_size']) / df_av1['input_size']) * 100
df_av1

,codec,options,input_size,output_size,evaluation_result,elapsed_time,crf,speed,bitrate,original_codec,diferenca_percentual_tamanho
0,av1,"{'crf': 30, 'speed': 8, 'bitrate': None, 'thre...",313907,466952,47.901434,452.920968,30,8,NaN,h264,48.754886
1,av1,"{'crf': 30, 'speed': 8, 'bitrate': None, 'thre...",314204,483673,46.953364,399.907560,30,8,NaN,h265,53.935978
2,av1,"{'crf': 30, 'speed': 8, 'bitrate': None, 'thre...",310009,497256,47.605341,381.202630,30,8,NaN,h264+,60.400505


In [40]:
tam_menor = df_av1['diferenca_percentual_tamanho'].min()
tam_menor_i = df_av1['diferenca_percentual_tamanho'].idxmin()
print(f"Maior diminuicao de tamaho: {tam_menor} na linha: {tam_menor_i}")

temp_menor = df_av1['elapsed_time'].min()
temp_menor_i = df_av1['elapsed_time'].idxmin()
print(f"Conversão mais rápida: {temp_menor} na linha: {temp_menor_i}")

psnr_maior = df_av1['evaluation_result'].max()
psnr_maior_i = df_av1['evaluation_result'].idxmax()
print(f"Maior valor de PSNR: {psnr_maior} na linha: {psnr_maior_i}")

psnr_menor = df_av1['evaluation_result'].min()
psnr_menor_i = df_av1['evaluation_result'].idxmin()
print(f"Menor valor de PSNR: {psnr_menor} na linha: {psnr_menor_i}")

Maior diminuicao de tamaho: 48.75488600126789 na linha: 0
Conversão mais rápida: 381.2026300430298 na linha: 2
Maior valor de PSNR: 47.901434 na linha: 0
Menor valor de PSNR: 46.953364 na linha: 1


### Sobre os Testes

A princípio, os primeiros testes foram feitos com os codecs VP8 e VP9 e levaram aproximadamente 10 e 20 minutos respectivamente para recodificarem 18 vídeos de 30 segundos cada um.  
Os testes com AV1 levaram 20 minutos para recodificarem 3 vídeos de 5 segundos.  
Isso significa que no tempo do codec VP9 processar um total de 540 segundos de vídeo o codec AV1 processou 15 segundos de vídeo. O encoder AV1 obteve os melhores valores nos parâmetros de qualidade, contudo um tempo mínimo 06:21 minutos a conversão para este codec é inviabilizada.

Dito isso, as três tabelas de tempos anteriormente expostas serão agora avaliadas buscando responder as questões que eram o objetivo deste exercício de pesquisa. 
 
---

Primeiramente, a maior diminuição percentual de tamanho foi alcançada pelo encoder VP8 com uma diminuição de 59,6% (743564bits) no teste com h264, crf=50 e speed=2. O encoder VP9 teve sua maior diminuição de tamanho com os mesmos parâmetros de teste (55,16%, 825874bits, uma diferença de 82MBytes). 

Para o encoder VP8 a conversão mais eficiente em tempo levou apenas 14,33 segundos, mas este tempo não é distante do tempo das outras conversões realizadas na velocidade 2, da mesma forma que as outras conversões em velocidade 4 levaram ao redor de 43,57s ~ 50,38s. Deve ser então estudado (mais adiante) se a perda em velocidade compensa o ganho de espaço de disco.  

O alcance de valores de PSNR para o encoder VP8 foi 30,45 ~ 32,16. Esta diferência é irrisória dentro do alcance que apresentaremos mais adiante.  

Para o encoder VP9 a conversão mais eficiente com tamanho demorou 76,78 segundos, o que não é bom se levarmos em consideração que houveram conversões com diminuição de tamanho semelhantes que ocorreram em menos tempo: 64,22s (54,37%, crf=50, speed=4, h264) e 63,24s (51,13%, crf=50, speed=4, h264+).  

Nosso alcance de PSNR foi de 38,9 ~ 45,1 para o encoder VP9.  

Segundo o WikiPedia:  
```
Typical values for the PSNR in lossy image and video compression are between 30 and 50 dB,
provided the bit depth is 8 bits, where higher is better.
```  

Assim sendo, uma variação de 6,2 pontos representa 12% do nosso intervalo ideal e portanto a diferênça entre as classificações dos vídeos não é grandemente significante, ainda mais quando nossos melhores exemplos no que tange tempo conseguiram scores de 39,26 (63,24s, 51,13% ganho de tamanho, crf=50, speed=4, h264+) e 39,06 (54,37% ganho de tamamho, crf=50, speed=4, h264).  

## Analises Finais  

Depois de uma análise inicial sobre os resultados, aqui seremos matemáticos. Visto que a diferença entre as qualidades dos vídeos em VP8 tem muito pouca diferença do pior para o melhor (apenas 1,71) este parâmetro não será levado em consideração. Já a diferença entre as qualidades entre os encoders VP8 e VP9 pode ser considerada significante e assim sendo iremos olhar para as diferenças de custo das melhores opções de ambos encoders antes de escolher a melhor delas.  

Primeiramente, os tempos de recuperacao de video para cada codec sao de 0,6min/min de video para H264, 0,4min/min de video parar H264+ e 0,3min/min de video para H265. Esses valores são combinados com o fato de que o custo para 1 hora de recuperação de vídeo é de R$0,001. O custo pago para armazenar os videos (em qualquer formato) é de R$0,001 por MB. Por fim, os custos de processamento (para conversão) são de R$0,001 por hora. Vamos esquematizar:  
  
1. Para encontrarmos o **custo de armazenamento** ($C_a$) dos vídeos ORIGINAIS basta multiplicar o **tempo de vídeo** ($t_v$) (usaremos 200 câmeras, 24hrs/dia, por 1 mês) pelo **tamanho em Mb/min** ($T_{Mb}$) (10, 5 e 2,5 para H264, H264+ e H265) e pelo **custo de armazenamento de cada MB** ($C_{aMb}$) (R$0,001).  
$$C_a = t_v · T_{Mb} · C_{aMb}$$  
2. Para encontrarmos o **custo de armazenamento** ($C_a$) dos vídeo CONVERTIDOS multiplicamos seus **pesos** ($P_{Mb}$) pelo **peso p/min deles** ($P_{min}$) (Mb/min, usaremos estimativa) pelo **custo de armazenamento de cada MB** ($C_{aMb}$).
$$C_a = P_{Mb} · P_{min} · C_{aMb}$$  
3. Para encontrarmos o **custo de se recuperar da base** ($C_{rb}$) os videos basta multiplicar o **tempo de vídeo** ($t_v$) (200 câmeras, 24hrs/dia, por 1 mês) pela **velocidade de recuperação** ($V_{rec}$) pelo **preço de recuperação** ($P_{rec}$).
$$C_{rb} = t_v · V_{rec} · P_{rec}$$  
4. Para calcularmos o **custo de conversão** ($C_{con}$) precisamos do **custo de se recuperar videos da base** ($C_{rb}$) + (**tempo de conversão** ($t_{con}$) multiplicado **pelo custo de conversão por minuto**($c_{con}$)) (RR$0,001).  
$$C_{con} = C_{rb} + (t_{con} · c_{con})$$

Dessa forma, buscamos fazer com que o **custo de conversão (4)** seja menor que o **custo de armazenamento dos originais (1)**.

---

O primeiro valor necessário (1) é o volume de armazenamento dos videos originais, para o tempo de video foi pedido usar era de 200 cameras gravando 24hrs por dia com dados armazenados na cloud por 1 mês: 
- 24 · 200 · 30 = 144.000 horas de gravação.
- 144.00 · 60 = 8.640.000 minutos de gravação por mês.
- 8.640.000 · 5 (5Mb/min) = 43.200.000Mb de armazenamento em H264+.
  - 43.200.000 · R$0,001 = **R$43.200 por mês em armazenamento dos originais em H264+**.
- 8.640.000 · 10 (10Mb/min) = 86.400.000Mb de armazenamento em H264.
  - 86.400.000 · R$0,001 = **R$83.400 por mês em armazenamento dos originais em H264**.
- 8.640.000 · 2,5 (2,5Mb/min)= 21.600.000Mb de armazenamento em H265.
  - 21.600.000 · R$0,001 = **R$21.600 por mês em armazenamento dos originais em H265**.

---  

Vamos agora encontrar o custo de armazenamento dos videos convertidos (2).  

**VP8**

In [74]:
df_custo_arm_vp8 = df_vp8[['original_codec', 'options', 'evaluation_result', 'elapsed_time','output_size']].copy()

df_custo_arm_vp8['custo_armazenamento'] = ((((df_custo_arm_vp8['output_size']) * 2) / 1000000) * 8640000) * 0.001
# os videos tem 30 segundos, precisa-se multiplicar este valor por 2 para que nosso tempo de conversao
# "equivalha" a conversao de um video de um minuto. Depois disso dividimos este valor por 1000000 para coloca-lo
# na base de MegaBytes. Multiplicaremos entao minutos de conversao por minutos de video a ser convertido
# e então este valor pelo custo da conversao, assim chegamos ao resultado.

df_custo_arm_vp8['custo_armazenamento'] = df_custo_arm_vp8['custo_armazenamento'].round(2)
df_custo_arm_vp8

,original_codec,options,evaluation_result,elapsed_time,output_size,custo_armazenamento
0,h264,"{'crf': 15, 'speed': 4, 'threads': 2}",30.492520,16.163413,1911505,33030.81
1,h264,"{'crf': 15, 'speed': 2, 'threads': 2}",32.160616,43.575843,749977,12959.60
2,h264,"{'crf': 30, 'speed': 4, 'threads': 2}",30.492520,15.600347,1911505,33030.81
3,h264,"{'crf': 30, 'speed': 2, 'threads': 2}",32.160616,45.654684,749977,12959.60
4,h264,"{'crf': 50, 'speed': 4, 'threads': 2}",30.492520,14.905854,1911505,33030.81
5,h264,"{'crf': 50, 'speed': 2, 'threads': 2}",32.135304,44.200587,743564,12848.79
6,h265,"{'crf': 15, 'speed': 4, 'threads': 2}",30.451200,14.582265,1778665,30735.33
7,h265,"{'crf': 15, 'speed': 2, 'threads': 2}",31.916114,49.724072,691894,11955.93
8,h265,"{'crf': 30, 'speed': 4, 'threads': 2}",30.451200,14.623293,1778665,30735.33
9,h265,"{'crf': 30, 'speed': 2, 'threads': 2}",31.916114,50.380263,691894,11955.93


**VP9**

In [75]:
df_custo_arm_vp9 = df_vp9[['original_codec', 'options', 'evaluation_result', 'elapsed_time','output_size']].copy()

df_custo_arm_vp9['custo_armazenamento'] = ((((df_custo_arm_vp9['output_size']) * 2) / 1000000) * 8640000) * 0.001
# os videos tem 30 segundos, precisa-se multiplicar este valor por 2 para que nosso tempo de conversao
# "equivalha" a conversao de um video de um minuto. Depois disso dividimos este valor por 1000000 para coloca-lo
# na base de MegaBytes. Multiplicaremos entao minutos de conversao por minutos de video a ser convertido
# e então este valor pelo custo da conversao, assim chegamos ao resultado.

df_custo_arm_vp9['custo_armazenamento'] = df_custo_arm_vp9['custo_armazenamento'].round(2)
df_custo_arm_vp9

,original_codec,options,evaluation_result,elapsed_time,output_size,custo_armazenamento
0,h264,"{'crf': 15, 'speed': 4, 'threads': 2}",44.966804,67.517135,4879147,84311.66
1,h264,"{'crf': 15, 'speed': 2, 'threads': 2}",45.032262,80.596506,4672195,80735.53
2,h264,"{'crf': 30, 'speed': 4, 'threads': 2}",43.510837,67.062914,2632118,45483.00
3,h264,"{'crf': 30, 'speed': 2, 'threads': 2}",43.692490,77.714066,2569960,44408.91
4,h264,"{'crf': 50, 'speed': 4, 'threads': 2}",39.066017,64.221460,840491,14523.68
5,h264,"{'crf': 50, 'speed': 2, 'threads': 2}",39.226264,76.787624,825874,14271.10
6,h265,"{'crf': 15, 'speed': 4, 'threads': 2}",43.291725,65.987115,4155697,71810.44
7,h265,"{'crf': 15, 'speed': 2, 'threads': 2}",43.332151,79.020004,4004487,69197.54
8,h265,"{'crf': 30, 'speed': 4, 'threads': 2}",42.337003,66.495231,2262748,39100.29
9,h265,"{'crf': 30, 'speed': 2, 'threads': 2}",42.462425,76.764789,2216712,38304.78


---

Buscaremos agora por (3). Os custos de recuperação são os seguintes:  

- (8.640.000 · 0,6) · R$0,001 = **R$5.184,00 por recuperação dos videos em H264**.
- (8.640.000 · 0,4) · R$0,001 = **R$3.456,00 por recuperação dos vídeos em H264+**.
- (8.640.000 · 0,3) · R$0,001 = **R$2.592,00 por recuperação dos vídeos em H265**.



---

Por fim, agora buscamos o preço de conversão para cada preset em cada um dos encoders(4).

**VP8**

In [76]:
df_custo_total_vp8 = df_custo_arm_vp8.copy()

df_custo_total_vp8['custo_processamento'] = ((((df_custo_total_vp8['elapsed_time']) * 2) / 60) * 8640000) * 0.001
# os videos tem 30 segundos, precisa-se multiplicar este valor por 2 para que nosso tempo de conversao
# "equivalha" a conversao de um video de um minuto. Depois disso dividimos este valor por 60 para coloca-lo
# na base de minutos. Multiplicaremos entao minutos de conversao por minutos de video a ser convertido
# e então este valor pelo custo da conversao, assim chegamos ao resultado.

# Soma-se os valores de recuperacao de video para cada codec
df_custo_total_vp8.loc[df_custo_total_vp8['original_codec'] == 'h264', 'custo_processamento'] += 5184
df_custo_total_vp8.loc[df_custo_total_vp8['original_codec'] == 'h264+', 'custo_processamento'] += 3456
df_custo_total_vp8.loc[new_df['original_codec'] == 'h265', 'custo_processamento'] += 2592

df_custo_total_vp8['custo_processamento'] = df_custo_total_vp8['custo_processamento'].round(2)


df_custo_total_vp8['custo_total'] = df_custo_total_vp8['custo_processamento'] + df_custo_total_vp8['custo_armazenamento']

df_custo_total_vp8

,original_codec,options,evaluation_result,elapsed_time,output_size,custo_armazenamento,custo_processamento,custo_total
0,h264,"{'crf': 15, 'speed': 4, 'threads': 2}",30.492520,16.163413,1911505,33030.81,9839.06,42869.87
1,h264,"{'crf': 15, 'speed': 2, 'threads': 2}",32.160616,43.575843,749977,12959.60,17733.84,30693.44
2,h264,"{'crf': 30, 'speed': 4, 'threads': 2}",30.492520,15.600347,1911505,33030.81,9676.90,42707.71
3,h264,"{'crf': 30, 'speed': 2, 'threads': 2}",32.160616,45.654684,749977,12959.60,18332.55,31292.15
4,h264,"{'crf': 50, 'speed': 4, 'threads': 2}",30.492520,14.905854,1911505,33030.81,9476.89,42507.70
5,h264,"{'crf': 50, 'speed': 2, 'threads': 2}",32.135304,44.200587,743564,12848.79,17913.77,30762.56
6,h265,"{'crf': 15, 'speed': 4, 'threads': 2}",30.451200,14.582265,1778665,30735.33,6791.69,37527.02
7,h265,"{'crf': 15, 'speed': 2, 'threads': 2}",31.916114,49.724072,691894,11955.93,16912.53,28868.46
8,h265,"{'crf': 30, 'speed': 4, 'threads': 2}",30.451200,14.623293,1778665,30735.33,6803.51,37538.84
9,h265,"{'crf': 30, 'speed': 2, 'threads': 2}",31.916114,50.380263,691894,11955.93,17101.52,29057.45


**VP9**

In [77]:
df_custo_total_vp9 = df_custo_arm_vp9.copy()

df_custo_total_vp9['custo_processamento'] = ((((df_custo_total_vp9['elapsed_time']) * 2) / 60) * 8640000) * 0.001
# os videos tem 30 segundos, precisa-se multiplicar este valor por 2 para que nosso tempo de conversao
# "equivalha" a conversao de um video de um minuto. Depois disso dividimos este valor por 60 para coloca-lo
# na base de minutos. Multiplicaremos entao minutos de conversao por minutos de video a ser convertido
# e então este valor pelo custo da conversao, assim chegamos ao resultado.

# Soma-se os valores de recuperacao de video para cada codec
df_custo_total_vp9.loc[df_custo_total_vp9['original_codec'] == 'h264', 'custo_processamento'] += 5184
df_custo_total_vp9.loc[df_custo_total_vp9['original_codec'] == 'h264+', 'custo_processamento'] += 3456
df_custo_total_vp9.loc[new_df['original_codec'] == 'h265', 'custo_processamento'] += 2592

df_custo_total_vp9['custo_processamento'] = df_custo_total_vp9['custo_processamento'].round(2)


df_custo_total_vp9['custo_total'] = df_custo_total_vp9['custo_processamento'] + df_custo_total_vp9['custo_armazenamento']

df_custo_total_vp9

,original_codec,options,evaluation_result,elapsed_time,output_size,custo_armazenamento,custo_processamento,custo_total
0,h264,"{'crf': 15, 'speed': 4, 'threads': 2}",44.966804,67.517135,4879147,84311.66,24628.93,108940.59
1,h264,"{'crf': 15, 'speed': 2, 'threads': 2}",45.032262,80.596506,4672195,80735.53,28395.79,109131.32
2,h264,"{'crf': 30, 'speed': 4, 'threads': 2}",43.510837,67.062914,2632118,45483.00,24498.12,69981.12
3,h264,"{'crf': 30, 'speed': 2, 'threads': 2}",43.692490,77.714066,2569960,44408.91,27565.65,71974.56
4,h264,"{'crf': 50, 'speed': 4, 'threads': 2}",39.066017,64.221460,840491,14523.68,23679.78,38203.46
5,h264,"{'crf': 50, 'speed': 2, 'threads': 2}",39.226264,76.787624,825874,14271.10,27298.84,41569.94
6,h265,"{'crf': 15, 'speed': 4, 'threads': 2}",43.291725,65.987115,4155697,71810.44,21596.29,93406.73
7,h265,"{'crf': 15, 'speed': 2, 'threads': 2}",43.332151,79.020004,4004487,69197.54,25349.76,94547.30
8,h265,"{'crf': 30, 'speed': 4, 'threads': 2}",42.337003,66.495231,2262748,39100.29,21742.63,60842.92
9,h265,"{'crf': 30, 'speed': 2, 'threads': 2}",42.462425,76.764789,2216712,38304.78,24700.26,63005.04


## Conclusão

Agora com todos os cálculos realizados somos capazes de aferir objetivamente qual o melhor escolha para codec original, qual a melhor escolha para codec final e qual melhor configuração de parâmetros.

**VP8**

In [82]:
process_menor = df_custo_total_vp8['custo_processamento'].min()
process_menor_i = df_custo_total_vp8['custo_processamento'].idxmin()
print(f"Menor custo de processamento: {process_menor} na linha: {process_menor_i}")

arm_menor = df_custo_total_vp8['custo_armazenamento'].min()
arm_menor_i = df_custo_total_vp8['custo_armazenamento'].idxmin()
print(f"Menor custo de armazenamento: {arm_menor} na linha: {arm_menor_i}")

custo_menor_vp8 = df_custo_total_vp8['custo_total'].min()
custo_menor_i = df_custo_total_vp8['custo_total'].idxmin()
print(f"Menor custo total: {custo_menor_vp8} na linha: {custo_menor_i}")

psnr_maior = df_custo_total_vp8['evaluation_result'].max()
psnr_maior_i = df_custo_total_vp8['evaluation_result'].idxmax()
print(f"Maior valor de PSNR: {psnr_maior} na linha: {psnr_maior_i}")

Menor custo de processamento: 6759.0 na linha: 10
Menor custo de armazenamento: 10497.76 na linha: 17
Menor custo total: 27137.58 na linha: 17
Maior valor de PSNR: 32.160616 na linha: 1


Levando em consideração os valores:  
- **R$43.200 por mês em armazenamento dos originais em H264+**.
- **R$83.400 por mês em armazenamento dos originais em H264**.
- **R$21.600 por mês em armazenamento dos originais em H265**.

A conversão de **H264+ para VP8** com parâmetros **crf = 50** e **speed = 2** rende uma diferença de **R$16.063,00** no primeiro mês de armazenamento (tendo pago também a conversão) e de **R$10.497,76** nos meses subsequentes onde só é necessário pagar pelo armazenamento. Além disso, o valor do PSNR para esta configuração de encoding é de 32.103204, uma diferença de 0,005 pontos do melhor resultado.  
A Conversão de **H264 para VP8** com parâmetros **crf = 50** e **speed = 2** rende uma diferença de **R$55.527,83** pagando o reencoding e de **R$12.848,79** por mês depois do processamento.  
A Conversão de **H265 para VP8** com parâmetros **crf = 50** e **speed = 2** rende uma diferença **extra** de **R$6,272,17** pagando o reencoding e de **R$11.978,79** por mês depois do processamento  em relação ao anterior.  


**VP9**

In [80]:
process_menor = df_custo_total_vp9['custo_processamento'].min()
process_menor_i = df_custo_total_vp9['custo_processamento'].idxmin()
print(f"Menor custo de processamento: {process_menor} na linha: {process_menor_i}")

arm_menor = df_custo_total_vp9['custo_armazenamento'].min()
arm_menor_i = df_custo_total_vp9['custo_armazenamento'].idxmin()
print(f"Menor custo de armazenamento: {arm_menor} na linha: {arm_menor_i}")

custo_menor_vp8 = df_custo_total_vp9['custo_total'].min()
custo_menor_i = df_custo_total_vp9['custo_total'].idxmin()
print(f"Menor custo total: {custo_menor_vp8} na linha: {custo_menor_i}")

psnr_maior = df_custo_total_vp9['evaluation_result'].max()
psnr_maior_i = df_custo_total_vp9['evaluation_result'].idxmax()
print(f"Maior valor de PSNR: {psnr_maior} na linha: {psnr_maior_i}")

Menor custo de processamento: 21025.64 na linha: 10
Menor custo de armazenamento: 11336.73 na linha: 17
Menor custo total: 33192.34 na linha: 16
Maior valor de PSNR: 45.173358 na linha: 13


Levando em consideração os valores:  
- **R$43.200 por mês em armazenamento dos originais em H264+**.
- **R$83.400 por mês em armazenamento dos originais em H264**.
- **R$21.600 por mês em armazenamento dos originais em H265**.

A conversão de **H264+ para VP9** com parâmetros **crf = 50** e **speed = 4** rende uma diferença de **R$10.007,76** no primeiro mês de armazenamento (tendo pago também a conversão) e de **R$11.520,39** nos meses subsequentes onde só é necessário pagar pelo armazenamento.  
A Conversão de **H264 para VP9** com parâmetros **crf = 50** e **speed = 4** rende uma diferença de **R$45.196,51** pagando o reencoding e de **R$14.523,69** por mês depois do processamento.  
A Conversão de **H265 para VP9** com parâmetros **crf = 50** e **speed = 4** rende uma diferença **extra** de **R$12.688** pagando o reencoding e de **R$13.262,49** por mês depois do processamento em relação ao anterior.  


Em resumo:
1. A melhor escolha entre H264, H264+ e H265 é o H265 pois é mais barato de se manter sem nenhum tipo de conversão. Deve ser considerado o fato de que ele não funciona em todos os browsers, o que pode causar problemas com clientes.
2. A melhor escolha para codec de destino é o VP8 devido ao fato de não possuir grande diferença de qualidade em relação ao VP9 (se mantém dentro do alcance de 30 e 50 de PSNR) e por ter custos de armazenamento e conversão muito reduzidos. Seriam necessários estudos para aferir o impacto de sua adoção nos modelos da empresa, visto que ele entrega qualidade menor.
3. A cofiguração de parâmetros a ser utilizada é a de Crf=50 e Speed=2 para conversão em VP8
4. A conversão de **H264+ para VP8** com parâmetros **crf = 50** e **speed = 2** rende uma diferença de **R$16.063,00** no primeiro mês de armazenamento (tendo pago também a conversão) e de **R$10.497,76** nos meses subsequentes onde só é necessário pagar pelo armazenamento.